# Transformer

本文主要参考以下资料来了解序列预测中state-of-the-art的Transformer：

- [The Illustrated Transformer](https://jalammar.github.io/illustrated-transformer/)
- [What is a Transformer?](https://medium.com/inside-machine-learning/what-is-a-transformer-d07dd1fbec04)

Transformer也是‘Attention Is All You Need’ 这篇文章中介绍的结构。类似LSTM，Transformer是一种通过两个部分(编码器和解码器)将一个序列转换为另一个序列的架构，但它不同于前面描述的/现有的序列到序列模型，它不依赖循环网络(GRU、LSTM等)。

到目前为止，循环网络是捕捉序列中及时相关性的最佳方法之一。然而，提出‘Attention Is All You Need’ 这篇论文的团队证明，只有注意力机制而没有任何RNN(递归神经网络)的架构可以改善翻译任务和其他任务的结果（水文上还得进一步确认）

Transformer的架构还是有些复杂的，一点点来了解。

首先，一个全景，把整个模型当作一个black box，在机器翻译中，就是输入一句话，进入 Transformer，然后输出一句话。

打开 Transformer，我们看到一个encoder组件，一个decoder组件，以及它们之间的连接。

![](pictures/QQ截图20210312170559.png)

编码组件是一堆编码器(比如下图有6个编码器叠在一起，数字6没有什么特别的，可以尝试其他的安排)。解码组件是由相同数量的解码器组成的堆栈。

![](pictures/QQ截图20210312170705.png)

所有编码器在结构上都是相同的，他们也不共享权重。每一个都由两部分组成：

![](pictures/QQ截图20210312170759.png)

编码器的输入首先通过一个self-attention层，这一层帮助编码器在对特定单词编码时查看输入句子中的其他单词。我们将在稍后进一步探讨self-attention。自注意层的输出输入前馈神经网络。完全相同的前馈网络被独立地应用于每个位置。解码器拥有这两个层，但在它们之间是一个注意力层，帮助解码器关注输入句子的相关部分(类似于之前attention一文中seq2seq模型中的注意力)。

![](pictures/QQ截图20210312171050.png)

现在我们已经了解了模型的主要组成部分，让我们开始看看各种向量/张量，以及它们如何在这些组成部分之间流动，从而将训练过的模型的输入转换为输出。

还是翻译的例子，首先，一个embedding算法将词转换为向量。每个词都被embedded 到一个大小512的向量。下面就简单的用盒子表示。

embedding只发生在最底部的编码器。所有编码器都有一个共同的抽象，那就是它们接收到一个向量列表，每个向量的大小都是512——在底部的编码器中，它就是word embeddings，但在其他编码器中，它直接就是其下层的编码器的输出。这个列表的大小是我们可以设置的超参数——基本上它就是我们的训练数据集中最长句子的长度。

在我们的输入序列中embedding 单词后，每一个单词都会流经编码器的两层。

![](pictures/QQ截图20210312172353.png)

这里我们开始看到Transformer的一个关键属性，即每个位置的单词在编码器中流过它自己的路径。在self-attention层中，这些路径之间存在依赖关系。然而，前馈层没有这些依赖关系，因此各种路径可以在流经前馈层时并行执行（这个并行也是它受欢迎的一个重要原因）。

接下来看看编码器的每个子层发生了什么。

正如我们已经提到的，编码器接收向量列表作为输入。它通过将这些向量传递到一个“自我注意”层，然后进入一个前馈神经网络，然后将输出向上发送到下一个编码器来处理这个列表。那么这个self-attention在做什么呢？

比如想要翻译这个句子：”The animal didn't cross the street because it was too tired”

这个句子中的“it”指的是什么?它指的是街道还是动物?这对人类来说是个简单的问题，但对算法来说就不那么简单了。

当模型在处理“it”这个词时，self-attention使它能把“it”和“animal”联系起来。

当模型处理每个单词(输入序列中的每个位置)时，“自我注意”允许它查看输入序列中的其他位置，以寻找有助于为这个单词找到更好编码的线索。

如果熟悉RNN，请考虑如何维护隐藏状态，使RNN能够将它处理过的以前的单词/向量表示与它正在处理的当前单词/向量表示结合起来。自我注意是Transformer将其他相关词的“理解”融入我们当前正在处理的词的方法。如下图，it能和前面的词建立不同程度的联系。

![](pictures/QQ截图20210312184558.png)

下面是向量实现的细节。

计算self-attention的第一步是从每个编码器的输入向量(在本例中是每个单词的embedding)中创建三个向量。对于每个单词，我们创建一个Query向量、一个Key向量和一个Value向量。这些向量是通过将嵌入的向量与我们在训练过程中训练过的三个矩阵相乘而得到的。

注意这些新的向量在维数上比 embedding 向量小。其维数为64，而embedding向量和编码器输入/输出向量的维数为512。它们不需要更小了，这是一种架构选择，以使多multiheaded 注意力(大部分)的计算保持恒定。

![](pictures/QQ截图20210312185101.png)

什么是“query”, “key”, 和 “value” 向量?

它们是对计算和思考attention很有用的抽象概念。继续阅读下面的“注意力是如何计算的”，就会知道所需要知道的关于这些向量所起的作用。

计算self-attention的第二步是计算分数。假设我们在计算这个例子中第一个单词“Thinking”的self-attention。我们需要给输入句子的每个单词打分。分数决定了当我们在一个特定的位置对一个单词进行编码时，在输入句子的其他部分需要关注多少。

分数是通过 query vector 和与我们正在评分的单词的 key vector 的点积来计算的。所以如果我们处理位置1的单词的 self-attention，第一个分数就是q1和k1的点积。第二个分数是q1和k2的点积。

![](pictures/QQ截图20210312185522.png)

第三步和第四步是将分数除以8(论文中使用的关键向量的维数64的平方根)。这能带来更稳定的梯度。这里可能有其他可能的值，不过这是个默认值)，然后通过softmax操作传递结果。Softmax将分数标准化，所以它们都是正的，加起来等于1。

![](pictures/QQ截图20210312185746.png)

这个softmax分数决定了每个单词在这个位置的表达量。显然，这个位置的单词的softmax分数最高，但有时注意与当前单词相关的另一个单词是有用的。

第五步是将每个值向量乘以softmax分数(准备将它们相加)。这里的直觉是保持我们想要关注的单词的值不变，并淹没不相关的单词(例如，通过将它们乘以很小的数字，如0.001)。

第六步是加权值向量的总和。这将在这个位置产生self-attention层的输出(对于第一个单词)。

![](pictures/self-attention-output.png)

self-attention的计算到此结束。得到的向量是一个我们可以发送给前馈神经网络的向量。然而，在实际实现中，为了更快地处理，这个计算是以矩阵形式进行的。现在让我们来看一下，我们已经看到了在单词水平上计算的直觉。

下面是 self-attention 的矩阵形式。

![](pictures/self-attention-matrix-calculation.png)

最后，由于我们处理的是矩阵，我们可以将步骤2到步骤6压缩到一个公式中来计算self-attention 层的输出。

![](pictures/self-attention-matrix-calculation-2.png)

继续增加一个叫做“multi-head”的attention机制来改进self-attention。这能从两个方面提高attention层的性能：

1. 扩展了模型关注不同位置的能力。在上面的示例中，z1包含了所有其他编码的一些内容，但它可能由实际的单词本身控制。如果我们要翻译这样一句话:The animal didn ' t cross The street because It was too tired，我们想知道“it”指的是哪个词。
2. 赋予attention层多个“表示子空间”。对于multi-head注意力，我们不仅有一个，而是有多个集合的查询/键/值权重矩阵(Transformer使用8个注意力头，所以我们最终为每个编码器/解码器有8个集合)。每个集合都是随机初始化的。然后，在训练之后，每个集合被用来将input embeddings(或来自较低编码器/解码器的向量)投影到不同的表示子空间。

![](pictures/transformer_attention_heads_qkv.png)

用上面提到的自我注意力计算方法，用不同的权重矩阵进行8次不同的计算，我们最终会得到8个不同的Z矩阵

![](pictures/transformer_attention_heads_z.png)

这给我们留下了一点挑战。前馈层不需要八个矩阵——它只需要一个矩阵(一个向量代表每个单词)。所以我们需要一种方法把这8个压缩成一个矩阵。

我们怎么做呢?我们将这些矩阵连接起来，然后将它们乘以一个额外的权重矩阵WO来做一次转换。

![](pictures/transformer_attention_heads_weight_matrix_o.png)

以上就是 multi-head self-attetion，还是蛮多内容，可以归纳到一张图：

![](pictures/transformer_multi-headed_self-attention-recap.png)